In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from datetime import date, datetime, timedelta, time
import pandas as pd
import seaborn
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import gridspec
import numpy as np

In [ ]:
import scipy
from scipy.stats import norm
import scipy.optimize as op
import emcee
import triangle

In [ ]:
import simtransient as st
import simtransient.utils as stutils
from simtransient import hammer
from simtransient.models.supernova import Sn1aOpticalEnsemble, Sn2OpticalEnsemble
from simtransient.modelrun import ModelRun

In [ ]:
# Commence plot tweaking!

rpal = seaborn.color_palette('Reds',7)
bpal = seaborn.color_palette('Blues',7)
gpal = seaborn.color_palette('Greens',5)
orpal = seaborn.color_palette('YlOrBr',7)
kpal = seaborn.color_palette('Greys',6)
for pal in (rpal, bpal, gpal, orpal, kpal):
    seaborn.palplot(pal)

In [ ]:
pal1 = {}
pal1['trace'] = bpal[3]
pal1['data']= gpal[-1]
pal1['true']=kpal[-2]
pal1['forecast']='k'
pal2 = pal1.copy()
pal2['trace']=rpal[3]

In [ ]:
nthreads=4

In [ ]:
emcee_kwargs=dict(threads=nthreads)
mr1 = ModelRun(ensemble=Sn1aOpticalEnsemble())
sampler1 = mr1.get_sampler(threads=nthreads)
mr2 = ModelRun(ensemble=Sn2OpticalEnsemble())
sampler2 = mr2.get_sampler(threads=nthreads)

In [ ]:
%%capture
mr1.run(sampler1,300)
mr2.run(sampler2,300)
print

In [ ]:
seaborn.set_context('poster')
tsteps=np.linspace(-30,80,500)
t_forecast=50
mr=mr1
maxprior_pars = mr.ensemble.gauss_pars.T.mu.copy()
maxprior_pars['t0']=0
maxprior = mr.ensemble.get_curve(**maxprior_pars)


gs = gridspec.GridSpec(3, 8)# width_ratios=[1, 1])
axes = plt.subplot(gs[:,:4]),plt.subplot(gs[:,4:])

mr1.plot_forecast(tsteps,
#                        t_forecast=t_forecast, 
                axes=(axes[0],None), 
                       palette=pal1
                 )

mr2.plot_forecast(tsteps, 
#                   t_forecast=t_forecast,
                  palette=pal2,
                  axes=(axes[1],None), 
                 )

axes[0].set_ylabel('Flux')
axes[1].set_yticklabels([])
axes[0].set_ylim(axes[1].get_ylim())
for ax in axes:
    ax.set_xlabel('Time')
    ax.set_ylim(0,27)
plt.gcf().tight_layout()
plt.savefig('prior_models.pdf')
